# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Instituto de Ciências Exatas e Tecnológicas
#### Profa. Andreza C. Beezão Moreira (DMM/UFLA)
#### Prof. Mayron César O. Moreira (DCC/UFLA)
#### Aluno: Bruno Crespo Ferreira

## Problema

A Empresa de Viação ao Aérea Brasileira está estudando a compra de três tipos de aviões: Boeing 717 para as pontes aéreas a curta distância, Boeing 737-500 para vôos domésticos e internacionais de média distância e MD-11 para vôos internacionais de longa distância. Em um estudo preliminar, considerou-se que a capacidade máxima dos aviões a serem comprados será sempre preenchida para efeito de planejamento. Os dados de planejamento constam na Tabela abaixo.

| Avião              | Custo (milhões de \$) | Receita Teórica (milhões de \$) | Pilotos aptos |
|--------------------|----------------------------|--------------------------------------|---------------|
| **BOEING 717**     | 5.1                        | 330                                  | 30            |
| **BOEING 737-500** | 3.6                        | 300                                  | 20            |
| **MD-11**          | 6.8                        | 420                                  | 10            |

**Tabela:** Informações sobre os aviões.

A verba disponível para as compras é de 220 milhões de dólares. Os pilotos de MD-11 podem pilotar todos os aviões da empresa, mas os demais pilotos só podem ser escalados às aeronaves a que foram habilitados. Cada aeronave necessita de dois pilotos para operar. As oficinas de manutenção podem suportar até 40 Boeings 717. Um Boeing 737-500 equivale, em esfoço de manutenção, a $3/4$, e um MD-11 a $5/3$, quando referidos ao Boeing 717.

### Objetivo

Apresente um modelo que otimize as aquisições de aviões nesta empresa, maximizando as receitas.

## Instalação da biblioteca PuLP

In [1]:
!pip install pulp
import pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 19.2 MB/s eta 0:00:00


## Parâmetros

* $A$: conjunto dos tipos de aviões disponíveis para compra.
* $c_{i} \in \mathbb{R}_+$: custo de compra do avião $i \in A$.
* $r_{i} \in \mathbb{R}_+$: receita teórica gerada pela compra do avião $i \in A$.
* $p_{i} \in \mathbb{N}$: quantidade de pilotos aptos a pilotar o avião $i \in A$.
* $o_{i} \in \mathbb{R}_+$: capacidade de manutenção (em relação ao Boeing 717) do avião $i \in A$.
* $verba \in \mathbb{R}_+$: verba total (em milhões) da empresa para a compra dos aviões.
* $capacidade\_manutencao \in \mathbb{N}$: capacidade de manutenção total de aviões (em relação ao Boeing 717) que a empresa suporta.

In [2]:
A = ['Boeing_717', 'Boeing_737_500', 'MD_11']

custos = {
    'Boeing_717': 5.1,
    'Boeing_737_500': 3.6,
    'MD_11': 6.8
}

receitas_teoricas = {
    'Boeing_717': 330.0,
    'Boeing_737_500': 300.0,
    'MD_11': 420.0
}

pilotos = {
    'Boeing_717': 30,
    'Boeing_737_500': 20,
    'MD_11': 10
}

manutencao = {    # Em relação ao Boeing 717
    'Boeing_717': 1,
    'Boeing_737_500': 3/4,
    'MD_11': 5/3
}

verba = 220.0

capacidade_manutencao = 40    # Em relação ao Boeing 717

## Declaração do objeto que representa o modelo matemático

In [3]:
modelo = pulp.LpProblem('compra_avioes', pulp.LpMaximize)

## Variáveis de decisão

* $x_{i} \ge 0$: quantidade de unidades compradas do avião $i \in A$

In [4]:
avioes_comprados = pulp.LpVariable.dicts('aviao', A, cat='Integer', lowBound=0)

## Função objetivo

* Maximizar as receitas das aquisições de aviões da empresa: $\max \sum_{i \in A} r_{i}x_{i}$

In [5]:
modelo += pulp.lpSum([receitas_teoricas[aviao] * avioes_comprados[aviao] for aviao in A])

## Restrições

* O custo de compra dos aviões não pode ultrapassar a verba da empresa: $\sum_{i \in A} c_{i}x_{i} \le verba$

In [6]:
modelo += pulp.lpSum([custos[aviao] * avioes_comprados[aviao] for aviao in A]) <= verba

* Cada avião precisa de 2 pilotos para operação;
* O piloto deve ser apto ao avião;
* A quantidade de pilotos necessários não pode ultrapassar a quantidade de pilotos aptos do avião.

Exceção: os pilotos do avião MD-11 podem pilotar todos os aviões da empresa.

**BOEING 717**

$2x_{1} \le p_{1} + p_{3}$

**BOEING 737-500**

$2x_{2} \le p_{2} + p_{3}$

**MD-11**

$2x_{3} \le p_{3}$

In [7]:
for aviao in avioes_comprados:
  if aviao != 'MD_11':
    modelo += 2*avioes_comprados[aviao] <= pilotos[aviao] + pilotos['MD_11']
  else:
    modelo += 2*avioes_comprados[aviao] <= pilotos[aviao]

* A quantidade de aviões não pode ultrapassar a capacidade das oficinas de manutenção: $\sum_{i \in A} o_{i}x_{i} \le capacidade\_manutencao$

In [8]:
modelo += pulp.lpSum([manutencao[aviao] * avioes_comprados[aviao] for aviao in A]) <= capacidade_manutencao

## Resolvendo o problema

* Mostrando o modelo (debug)

In [9]:
modelo

compra_avioes:
MAXIMIZE
330.0*aviao_Boeing_717 + 300.0*aviao_Boeing_737_500 + 420.0*aviao_MD_11 + 0.0
SUBJECT TO
_C1: 5.1 aviao_Boeing_717 + 3.6 aviao_Boeing_737_500 + 6.8 aviao_MD_11 <= 220

_C2: 2 aviao_Boeing_717 <= 40

_C3: 2 aviao_Boeing_737_500 <= 30

_C4: 2 aviao_MD_11 <= 10

_C5: aviao_Boeing_717 + 0.75 aviao_Boeing_737_500 + 1.66666666667 aviao_MD_11
 <= 40

VARIABLES
0 <= aviao_Boeing_717 Integer
0 <= aviao_Boeing_737_500 Integer
0 <= aviao_MD_11 Integer

* Resolvendo o modelo

In [10]:
status = modelo.solve()

## Imprimindo as soluções do problema

In [11]:
print('Status:', pulp.LpStatus[status])
print('Função objetivo:', f'O total de receita previsto será de {modelo.objective.value()} milhões de dólares no melhor caso.')

print('\n\t\tSoluções')
for aviao in avioes_comprados:
  print(f'Quantidade de {aviao} comprados:', avioes_comprados[aviao].value())
  print(f'Receita prevista: {avioes_comprados[aviao].value() * receitas_teoricas[aviao]} milhões de dólares.\n')

Status: Optimal
Função objetivo: O total de receita previsto será de 13200.0 milhões de dólares no melhor caso.

		Soluções
Quantidade de Boeing_717 comprados: 20.0
Receita prevista: 6600.0 milhões de dólares.

Quantidade de Boeing_737_500 comprados: 15.0
Receita prevista: 4500.0 milhões de dólares.

Quantidade de MD_11 comprados: 5.0
Receita prevista: 2100.0 milhões de dólares.

